In [1]:
import openai
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

# Initialize Embeddings and Vector Database
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.load_local("faiss_index", embedding_model)
retriever = vectorstore.as_retriever()

# Define a Critic Agent for Self-Evaluation
def evaluate_response(response, retrieved_docs):
    """Evaluate response correctness based on retrieved docs."""
    evaluation_prompt = f"""
    Given the retrieved context:
    {retrieved_docs}

    Assess the factual accuracy of the generated response:
    {response}

    If inaccurate, suggest a refined query to improve retrieval.
    """
    eval_response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a fact-checking AI."},
                  {"role": "user", "content": evaluation_prompt}]
    )
    return eval_response["choices"][0]["message"]["content"]

# Define the Agentic Corrective RAG Process
def agentic_corrective_rag(query):
    """Main pipeline for Agentic Corrective RAG."""
    retrieved_docs = retriever.get_relevant_documents(query)

    # First-pass response
    qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-4"), retriever=retriever)
    response = qa_chain.run(query)

    # Self-evaluation and correction
    feedback = evaluate_response(response, retrieved_docs)
    if "suggest" in feedback.lower():
        refined_query = feedback.split(":")[-1].strip()
        retrieved_docs = retriever.get_relevant_documents(refined_query)
        response = qa_chain.run(refined_query)

    return response

# Example Usage
query = "What are the latest developments in AI for medical diagnostics?"
corrected_response = agentic_corrective_rag(query)
print(corrected_response)
